In [1]:
import cmipld
import importlib
from collections import OrderedDict


repo_url = cmipld.utils.git.url()
io_url = cmipld.utils.git.url2io(repo_url)

whoami = cmipld.reverse_mapping()[io_url]
print('-'*50)
print(f'Parsing repo: {whoami}')
print(f'Location: {repo_url}')
print(f'Github IO link: {io_url}')
print('-'*50)

'''
To keep this set of functions generic, we create separate files containing all relevant functions for a repository to be processed. 
When running this script we determine the parent repository, and consequently its prefix. This is then used to grab all relevant functions for processing. 

Example usage: getattr(this,'activity')({'activity':[{"label":1,"id":'one'}]})

'''
# Dynamically import the target module and assign it to a variable
module_path = f"cmipld.cvs.{whoami}" # go to CMIPLD/CVs/<value of whoami>.py for functions
this = importlib.import_module(module_path)



make test for url2io
--------------------------------------------------
Parsing repo: cmip6plus
Location: https://github.com/WCRP-CMIP/CMIP6Plus_CVs
Github IO link: https://wcrp-cmip.github.io/CMIP6Plus_CVs/
--------------------------------------------------


/Users/daniel.ellis/customlib/homebrew/Caskroom/mambaforge/base/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Set up offline override for the current repository

from cmipld.utils.offline import LD_server

ldpath = cmipld.utils.git.ldpath()


# create a temporary directory containing (a copy of) the current repository
# localserver = LD_server(copy = [[ldpath,'current_repo']])
repos= { 
        'https://wcrp-cmip.github.io/WCRP-universe/': 'universal',
        'https://wcrp-cmip.github.io/MIP-variables/': 'variables',
        'https://wcrp-cmip.github.io/CMIP6Plus_CVs/': 'cmip6plus'
        }

localserver = LD_server(repos = repos.items(),copy = [[ldpath,whoami]],override='y')

# localserver = LD_server(copy = [[ldpath,'cmip6plus']])


localhost = localserver.start_server(8089)


Cloning into '/var/folders/hc/s_7lggq12nndglbdyrn3f91m1l58yd/T/cmipld_local_44h8v8902025-02-04T21:30:08/universal'...


https://wcrp-cmip.github.io/WCRP-universe/ universal /var/folders/hc/s_7lggq12nndglbdyrn3f91m1l58yd/T/cmipld_local_44h8v8902025-02-04T21:30:08


https://wcrp-cmip.github.io/MIP-variables/ variables /var/folders/hc/s_7lggq12nndglbdyrn3f91m1l58yd/T/cmipld_local_44h8v8902025-02-04T21:30:08


Cloning into '/var/folders/hc/s_7lggq12nndglbdyrn3f91m1l58yd/T/cmipld_local_44h8v8902025-02-04T21:30:08/variables'...




https://wcrp-cmip.github.io/CMIP6Plus_CVs/ cmip6plus /var/folders/hc/s_7lggq12nndglbdyrn3f91m1l58yd/T/cmipld_local_44h8v8902025-02-04T21:30:08


Cloning into '/var/folders/hc/s_7lggq12nndglbdyrn3f91m1l58yd/T/cmipld_local_44h8v8902025-02-04T21:30:08/cmip6plus'...




Repositories cloned into <TemporaryDirectory '/var/folders/hc/s_7lggq12nndglbdyrn3f91m1l58yd/T/cmipld_local_44h8v8902025-02-04T21:30:08'>
Copying the repo into LocalServer  /Users/daniel.ellis/WIPwork/CMIP6Plus_CVs/src-data/ --> cmip6plus
Repositories copied into <TemporaryDirectory '/var/folders/hc/s_7lggq12nndglbdyrn3f91m1l58yd/T/cmipld_local_44h8v8902025-02-04T21:30:08'>
Created SSL certificates in: /var/folders/hc/s_7lggq12nndglbdyrn3f91m1l58yd/T/cmipld_local_44h8v8902025-02-04T21:30:08
Serving /var/folders/hc/s_7lggq12nndglbdyrn3f91m1l58yd/T/cmipld_local_44h8v8902025-02-04T21:30:08 at https://localhost:8089


.+...............+...+.+......+..+...+....+...+...+...+......+.....+.+..+.+....................+....+......+...........+......+....+...............+........+......+....+..+...+.+......+.....+.+........+.......+..+...+....+...+..+.+.....+...+....+...+.........+........+.+..+.......+...+.........+++++++++++++++++++++++++++++++++++++++++++++*.....+...+.+.....+............+...+......+....+.....+.......+.........+...+..+.........+.+........+.+..+....+++++++++++++++++++++++++++++++++++++++++++++*........+................+...+.....+......+.+.........+........+.+......+....................+...............+...+.+...........+...+++++
....+........+......+....+..+.......+.....+...+...+.+...+...+++++++++++++++++++++++++++++++++++++++++++++*.............+..+....+++++++++++++++++++++++++++++++++++++++++++++*....+....+..........................+........................+.+............+...+...+......+...+......+...............+.....+.......+...+.....+.......+..+............+...+.......+......+...+.....

In [3]:
cmipld.processor.replace_loader(localhost,[list(i) for i in repos.items()]) 

Setting up location forwarding for:
 -  https://wcrp-cmip.github.io/WCRP-universe/ >>> https://localhost:8089/universal/ 

 -  https://wcrp-cmip.github.io/MIP-variables/ >>> https://localhost:8089/variables/ 

 -  https://wcrp-cmip.github.io/CMIP6Plus_CVs/ >>> https://localhost:8089/cmip6plus/ 



In [4]:

CV = OrderedDict()
cmipld.processor.clear_cache()


es_data = {}

In [5]:


es_data['project'] = project = cmipld.processor.EmbeddedFrame(f'{io_url}project/graph.jsonld') 

for item in [
    'repo',
    'mip-era',
    'license',
    'activity',
    'drs',
    'product',
    'required-global-attributes',
    'tables',
    ]:
    
    print(item)
    # CV[item] = getattr(locals(),item)
    # ld  = cmipld.processor.expand_document(f'{io_url}project/{item}.json', depth=3)[0]
    # functions may be autogenrated in (reponame).py under the CVs dir. 
    
    result = project.frame({'id':f'{io_url}project/{item}-list'})
    # from pprint import pprint 
    # pprint(result)
    
    # print(item,getattr(this,item))
    
    item = item.replace('-','_')
    CV[item] = getattr(this,item)(result[0])
    

127.0.0.1 - - [04/Feb/2025 21:30:10] "GET /cmip6plus/project/graph.jsonld HTTP/1.1" 200 -
127.0.0.1 - - [04/Feb/2025 21:30:10] "GET /universal/activity/_context_ HTTP/1.1" 200 -
127.0.0.1 - - [04/Feb/2025 21:30:10] "GET /universal/_context_ HTTP/1.1" 200 -
127.0.0.1 - - [04/Feb/2025 21:30:10] "GET /universal/license/_context_ HTTP/1.1" 200 -
127.0.0.1 - - [04/Feb/2025 21:30:11] "GET /universal/mip/_context_ HTTP/1.1" 200 -
127.0.0.1 - - [04/Feb/2025 21:30:11] "GET /universal/resolution/_context_ HTTP/1.1" 200 -
127.0.0.1 - - [04/Feb/2025 21:30:11] "GET /universal/product/_context_ HTTP/1.1" 200 -
127.0.0.1 - - [04/Feb/2025 21:30:11] "GET /variables/tables/_context_ HTTP/1.1" 200 -
127.0.0.1 - - [04/Feb/2025 21:30:11] "GET /variables/_context_ HTTP/1.1" 200 -
127.0.0.1 - - [04/Feb/2025 21:30:11] "GET /variables/variables/_context_ HTTP/1.1" 200 -
127.0.0.1 - - [04/Feb/2025 21:30:13] "GET /universal/product/graph.jsonld HTTP/1.1" 200 -
127.0.0.1 - - [04/Feb/2025 21:30:13] "GET /universal

 Please check output - this is not     corrected for secondary nested items.
[4] https://wcrp-cmip.github.io/CMIP6Plus_CVs/project ['https://wcrp-cmip.github.io/WCRP-universe/license/graph.jsonld', 'https://wcrp-cmip.github.io/WCRP-universe/activity/graph.jsonld', 'https://wcrp-cmip.github.io/WCRP-universe/product/graph.jsonld', 'https://wcrp-cmip.github.io/MIP-variables/tables/graph.jsonld', 'https://wcrp-cmip.github.io/CMIP6Plus_CVs/project/graph.jsonld', 'https://wcrp-cmip.github.io/WCRP-universe/mip/graph.jsonld']


100%|██████████| 6/6 [00:00<00:00, 45.34it/s]


repo
mip-era
license
activity
drs
product
required-global-attributes
tables


In [6]:

# adding the index properties
print('index')
CV.update(cmipld.processor.get(f'{io_url}/project/index-list.json')[0]['index'])

index


In [7]:
print('make a file for relevant realms and relevant realms')

for universe_item in ['realm','frequency']:
    # 'grid-label''nominal'
    print(universe_item)
    path = f'{cmipld.mapping["universal"]}{universe_item}/graph.jsonld'
    ld  = cmipld.jsonld.compact(path,path)['@graph']
    CV[universe_item] = getattr(this,universe_item)(ld)

make a file for relevant realms and relevant realms
realm
frequency


In [8]:
# path = f'{cmipld.mapping["universal"]}resolution/graph.jsonld'
# data  = cmipld.jsonld.compact(path,path)['@graph']

# CV['nominal_resolution'] = [f"{r['value']} {r['unit']}" for r in sorted(data, key=lambda x: float(x['value']) if x['value'].replace('.', '', 1).isdigit() else float('inf'))]

print('make a file for relevant nominal resolution')

make a file for relevant nominal resolution


In [ ]:
# url = cmipld.processor.resolve_prefix(file)

# loads the experiments graph and relevant dependancies. 
# cmipld.processor.clear_cache()
# localserver.copy_existing_repos([[ldpath,whoami]],override=True)
print(ldpath,whoami)

es_data = {}

for value in ['experiment', 'source']:
    # print(value)
    es_data[value] = cmipld.processor.EmbeddedFrame(f'{io_url}{value}/graph.jsonld') 
    result = es_data[value].frame({'@type':[f'wcrp:{value}']})
    CV[value] = getattr(this,universe_item)(result)
    
    

/Users/daniel.ellis/WIPwork/CMIP6Plus_CVs/src-data/ cmip6plus
 Please check output - this is not     corrected for secondary nested items.
[0] https://wcrp-cmip.github.io/CMIP6Plus_CVs/experiment ['https://wcrp-cmip.github.io/CMIP6Plus_CVs/experiment/graph.jsonld', 'https://wcrp-cmip.github.io/WCRP-universe/activity/graph.jsonld', 'https://wcrp-cmip.github.io/WCRP-universe/source-type/graph.jsonld', 'https://wcrp-cmip.github.io/CMIP6Plus_CVs/sub-experiment/graph.jsonld']


  0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
org = getattr(this,'organisation')(es_data['source'].frame({'@type':f'wcrp:source',"@explicit":True, 'organisation':{}}))
CV['institute_id'] = {k: v for d in org for k, v in d.items()}
print('organisations')

In [ ]:
es_data['source'].frame({'@type':f'wcrp:source',"@explicit":True, 'organisation':{}})

In [ ]:
CV['institution_id']

In [ ]:
# import glob,json
# files = glob.glob('../../../WCRP-universe/src-data/consortium/*.json')

# for f in files:
#     print(f)
#     data = json.load(open(f))
#     data['type'] = list(set(data['type']+['wcrp:organisation']))
    
#     json.dump(data,open(f,'w'),indent=4)

# data